In [1]:
## Installing packages
import pandas as pd
import numpy as np

In [2]:
## Importing dataset
df = pd.read_csv(r"IMDBdata_MainData.csv")

In [3]:
## Checking column names 
df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings.Source', 'Ratings.Value', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response', 'tomatoURL'],
      dtype='object')

In [4]:
## Checking for duplicated 
def duplicates_check(df):
    duplicated_record = df.duplicated().sum()
    return print(f"Number of duplicated records are: {duplicated_record}")
duplicates_check(df)

Number of duplicated records are: 0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5273 entries, 0 to 5272
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Title           5273 non-null   object 
 1   Year            5273 non-null   int64  
 2   Rated           4815 non-null   object 
 3   Released        5235 non-null   object 
 4   Runtime         5257 non-null   object 
 5   Genre           5271 non-null   object 
 6   Director        5272 non-null   object 
 7   Writer          5221 non-null   object 
 8   Actors          5266 non-null   object 
 9   Plot            5262 non-null   object 
 10  Language        5262 non-null   object 
 11  Country         5271 non-null   object 
 12  Awards          4462 non-null   object 
 13  Poster          5260 non-null   object 
 14  Ratings.Source  5273 non-null   object 
 15  Ratings.Value   5273 non-null   object 
 16  Metascore       3876 non-null   float64
 17  imdbRating      5273 non-null   f

In [6]:
def cleaning_dataset(df):

    columns_to_drop = ['tomatoURL', 'Website', 'Response', 'Poster','Ratings.Source', 'Plot', 'Released','DVD','Rated','Type','Ratings.Value']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')
    
    # Ensure 'Runtime' column is treated as a string and process it
    if 'Runtime' in df.columns:
        df['Runtime'] = df['Runtime'].astype(str).str.replace(' min', '', regex=False).str.strip()
        df['Runtime'] = pd.to_numeric(df['Runtime'], errors='coerce')  # Convert to numeric, handling invalid entries as NaN
        df['Runtime'] = df['Runtime'].fillna(0).astype(int)  # Replace null values with 0 and convert to integer
    
    # Ensure 'BoxOffice' column is treated as a string and process it
    if 'BoxOffice' in df.columns:
        df['BoxOffice'] = df['BoxOffice'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip()
        df['BoxOffice'] = pd.to_numeric(df['BoxOffice'], errors='coerce')  # Convert to numeric, handling invalid entries as NaN
        df['BoxOffice'] = df['BoxOffice'].fillna(0).astype(int)  # Replace null values with 0 and convert to integer
    
    # Handle 'Metascore' column
    if 'Metascore' in df.columns:
        df['Metascore'] = pd.to_numeric(df['Metascore'], errors='coerce')  # Convert to numeric, handling invalid entries as NaN
        df['Metascore'] = df['Metascore'].fillna(0).astype(int)  # Replace null values with 0 and convert to integer

    # Ensure 'imdbVotes' column is processed correctly
    if 'imdbVotes' in df.columns:
        df['imdbVotes'] = df['imdbVotes'].astype(str).str.replace(',', '', regex=False).str.strip()  # Remove commas

    # Replace missing values in 'Writer', 'Actors', 'Language', and 'Country' with 'Not Available'
    for column in ['Writer', 'Actors', 'Language', 'Country', 'Production','Genre','Director']:
        if column in df.columns:
            df[column] = df[column].fillna('Not Available')

    # Drop the 'Metascore' column if it exists - It has ~97% missing values
    if 'Metascore' in df.columns:
        df = df.drop(columns=['Metascore'])

    # Drop the 'Awards' column if it exists - Free form column
    if 'Awards' in df.columns:
        df = df.drop(columns=['Awards'])

    # Create a new 'Decade' column based on 'Year'
    if 'Year' in df.columns:
        df['Year'] = pd.to_numeric(df['Year'], errors='coerce')  # Convert to numeric, handling invalid entries as NaN
        df['Decade'] = df['Year'].apply(lambda x: (x // 10) * 10 if pd.notnull(x) else None)

    return df

# Apply the updated function to the dataset
df = cleaning_dataset(df)

In [ ]:
def cleaning_dataset_categories(df):
    """
    Splits the 'Language' and 'Genre' columns into individual categories and reshapes the table vertically,
    creating a row for each language and genre associated with a movie.

    Parameters:
    df (pd.DataFrame): The input dataframe.

    Returns:
    pd.DataFrame: The updated dataframe with one row per language and genre per movie.
    """
    # Process Language column
    if 'Language' in df.columns:
        df['Language'] = df['Language'].str.split(',')
        df = df.explode('Language')
        df['Language'] = df['Language'].str.strip()  # Remove extra spaces

    # Process Genre column
    if 'Genre' in df.columns:
        df['Genre'] = df['Genre'].str.split(',')
        df = df.explode('Genre')
        df['Genre'] = df['Genre'].str.strip()  # Remove extra spaces

    # Process Country column
    if 'Country' in df.columns:
        df['Country'] = df['Country'].str.split(',')
        df = df.explode('Country')
        df['Country'] = df['Country'].str.strip()  # Remove extra spaces

    # Process Director column
    if 'Director' in df.columns:
        df['Director'] = df['Director'].str.split(',')
        df = df.explode('Director')
        df['Director'] = df['Director'].str.strip()  # Remove extra spaces

    # Process Writer column
    if 'Writer' in df.columns:
        df['Writer'] = df['Writer'].str.split(',')
        df = df.explode('Writer')
        df['Writer'] = df['Writer'].str.strip()  # Remove extra spaces

    # Process Writer column
    if 'Actors' in df.columns:
        df['Actors'] = df['Actors'].str.split(',')
        df = df.explode('Actors')
        df['Actors'] = df['Actors'].str.strip()  # Remove extra spaces

    return df

# Apply the function to the dataset
df = cleaning_dataset_categories(df)